In [61]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

In [87]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [9]:
os.getcwd()

'/Users/valkoiset/Desktop/Python/Datacamp/ML with Python'

In [54]:
df = pd.read_csv('data/heart.csv')

In [100]:
# df.info()
# df
!pip install nbconvert

In [56]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [57]:
categorical = ['thal', 'ca', 'restecg', 'fbs', 'cp', 'sex']
df[categorical] = df[categorical].astype('category')

In [58]:
df.dtypes

age            int64
sex         category
cp          category
trestbps       int64
chol           int64
fbs         category
restecg     category
thalach        int64
exang          int64
oldpeak      float64
slope          int64
ca          category
thal        category
target         int64
dtype: object

In [59]:
df_dummies = pd.get_dummies(df)

In [64]:
df_dummies

,age,trestbps,chol,thalach,exang,oldpeak,slope,target,sex_0,sex_1,...,restecg_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3
0,63,145,233,150,0,2.3,0,1,0,1,...,0,1,0,0,0,0,0,1,0,0
1,37,130,250,187,0,3.5,0,1,0,1,...,0,1,0,0,0,0,0,0,1,0
2,41,130,204,172,0,1.4,2,1,1,0,...,0,1,0,0,0,0,0,0,1,0
3,56,120,236,178,0,0.8,2,1,0,1,...,0,1,0,0,0,0,0,0,1,0
4,57,120,354,163,1,0.6,2,1,1,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,140,241,123,1,0.2,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1
299,45,110,264,132,0,1.2,1,0,0,1,...,0,1,0,0,0,0,0,0,0,1
300,68,144,193,141,0,3.4,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1
301,57,130,131,115,1,1.2,1,0,0,1,...,0,0,1,0,0,0,0,0,0,1


In [97]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [98]:
df_dummies.describe()

,age,trestbps,chol,thalach,exang,oldpeak,slope,target,sex_0,sex_1,...,restecg_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,...,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,131.623762,246.264026,149.646865,0.326733,1.039604,1.399340,0.544554,0.316832,0.683168,...,0.013201,0.577558,0.214521,0.125413,0.066007,0.016502,0.006601,0.059406,0.547855,0.386139
std,9.082101,17.538143,51.830751,22.905161,0.469794,1.161075,0.616226,0.498835,0.466011,0.466011,...,0.114325,0.494765,0.411169,0.331734,0.248704,0.127605,0.081110,0.236774,0.498528,0.487668
min,29.000000,94.000000,126.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,120.000000,211.000000,133.500000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,55.000000,130.000000,240.000000,153.000000,0.000000,0.800000,1.000000,1.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,61.000000,140.000000,274.500000,166.000000,1.000000,1.600000,2.000000,1.000000,1.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,77.000000,200.000000,564.000000,202.000000,1.000000,6.200000,2.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [70]:
X = df_dummies.drop('target', axis=1)
y = df_dummies['target']

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [84]:
log = LogisticRegression(solver='warn')

In [85]:
df.shape

(303, 14)

In [88]:
log.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [92]:
y_pred = log.predict(X_test)

In [96]:
log.score(X_test, y_test)

0.8461538461538461